<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/vision3d/ICON_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

#ICON: Implicit Clothed humans Obtained from Normals
![](https://icon.is.tue.mpg.de/media/upload/teaser.jpg)

Made by [![Follow](https://img.shields.io/twitter/follow/yuliangxiu?style=social)](https://twitter.com/yuliangxiu)

Code in [![GitHub stars](https://img.shields.io/github/stars/yuliangxiu/ICON?style=social)](https://github.com/yuliangxiu/ICON)

HuggingFace Space in [![Hugging Face Spaces](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Spaces-yellow)](https://huggingface.co/spaces/Yuliang/ICON)

Chatroom at [![](https://img.shields.io/badge/Discord-Room-purple?style=flat&logo=Discord&logoColor=purple)](https://discord.gg/Vqa7KBGRyk)

Page at [![](https://img.shields.io/badge/Project-Page-blue?style=flat&logo=Google%20chrome&logoColor=blue)](https://icon.is.tue.mpg.de/)

</center>

In [ ]:
import IPython

IPython.display.HTML('<h2>5-Minute Presentation</h2><iframe width="960" height="480" src="https://www.youtube.com/embed/hZd6AYin2DE" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [ ]:
!nvidia-smi

Thu Jan  5 08:26:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**First of all, Edit > Notebook settings > Hardware Accelerator > Select "GPU"**

Current Colab works perfect on GPUs offered by free Colab (Tesla T4 usually), but failed on Colab Pro+ with better GPUs (Tesla V100)

In [ ]:
# get the code of ICON
!git clone https://github.com/YuliangXiu/ICON.git

Cloning into 'ICON'...
remote: Enumerating objects: 2488, done.
remote: Counting objects: 100% (2488/2488), done.
remote: Compressing objects: 100% (781/781), done.
remote: Total 2488 (delta 1715), reused 2428 (delta 1682), pack-reused 0
Receiving objects: 100% (2488/2488), 155.42 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (1715/1715), done.


In [ ]:
# package installation (will take ~15 min, just get a coffee)
from IPython.display import clear_output 
%cd /content/ICON
!bash colab.sh
clear_output()

## Now, you need to register **ICON**'s website

https://icon.is.tue.mpg.de/

## Then, please also register other dependencies: **SMPL, SMPLIFY**

![](https://raw.githubusercontent.com/YuliangXiu/ICON/master/assets/register.png)

In [ ]:
# download models and extra data (register required)
# skip download of "training", "pare", "pixie", 'hybrid' because the inference doesn't need them

# In short, input the username & password ONCE, and then always input "N" till the end

%cd /content/ICON
!bash fetch_data.sh # input "N" to avoid downloading data used for training
!bash fetch_hps.sh # input "N" to avoid downloading PARE, PIXIE or HybrIK
clear_output()

In [ ]:
!wget https://download.is.tue.mpg.de/icon/SMPL+X.zip --no-check-certificate -O ./data/thuman2/SMPL+X.zip
!unzip ./data/thuman2/SMPL+X.zip -d ./data/thuman2/
!rm ./data/thuman2/SMPL+X.zip

--2023-01-05 09:00:47--  https://download.is.tue.mpg.de/icon/SMPL+X.zip
Resolving download.is.tue.mpg.de (download.is.tue.mpg.de)... 192.124.27.139
Connecting to download.is.tue.mpg.de (download.is.tue.mpg.de)|192.124.27.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 304114160 (290M) [application/zip]
Saving to: ‘./data/thuman2/SMPL+X.zip’

./data/thuman2/SMPL 100%[===================>] 290.03M  12.5MB/s    in 22s     

2023-01-05 09:01:11 (12.9 MB/s) - ‘./data/thuman2/SMPL+X.zip’ saved [304114160/304114160]

Archive:  ./data/thuman2/SMPL+X.zip
   creating: ./data/thuman2/smpl/
  inflating: ./data/thuman2/smpl/0306.obj  
  inflating: ./data/thuman2/smpl/0461.obj  
  inflating: ./data/thuman2/smpl/0431.obj  
  inflating: ./data/thuman2/smpl/0008.obj  
  inflating: ./data/thuman2/smpl/0066.obj  
  inflating: ./data/thuman2/smpl/0495.obj  
  inflating: ./data/thuman2/smpl/0131.obj  
  inflating: ./data/thuman2/smpl/0294.obj  
  inflating: ./data/thuman2/smpl

In [ ]:
!source activate icon && pip install -r requirements.txt --use-deprecated=legacy-resolver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-5_5u8o3p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-5_5u8o3p
  Running command git checkout -q 3145dd4d16edaceb394838364b8e87a440f83c10
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 3145dd4d16edaceb394838364b8e87a440f83c10
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/YuliangXiu/neural_voxelization_layer.git to /tmp/pip-req-build-ifpkrbh6
  Running command git clone --filter=blob:none --quiet https://github.com/YuliangXiu/neural_voxelization_layer.git /tmp/pip-req-build-ifpkrbh6
  Resolved https://github.com/YuliangXiu/neural_voxelization_layer.git to commit b3c054e378fcca4daf6a53aceeee7ef39e86318b
  Preparing metadata (setup.py) ... done
  Cloning https://git

In [ ]:
!source activate icon && python scripts/render_single.py

Traceback (most recent call last):
  File "scripts/render_single.py", line 1, in <module>
    import lib.renderer.opengl_util as opengl_util
ModuleNotFoundError: No module named 'lib'


In [ ]:
%cd /content/ICON
!source activate icon && bash scripts/vis_batch.sh gen all

If you want to test on your own images, `choose files` and upload, otherwise, choose `cancel upload`, then ICON will run reconstruction on provided example image

In [ ]:
# run the test on examples
import os
from google.colab import files

model = 'icon-filter'                     # icon-filter / pifu / pamir

try:
  image_folder = '/content/sample_data'
  %cd /content/sample_data
  filename = list(files.upload().keys())[0] # update your full-body photo
except:
  image_folder = '/content/ICON/examples'
  filename = "22097467bffc92d4a5c4246f7d4edb75.png"

%cd /content/ICON/

!source activate icon && python -m apps.infer -cfg ./configs/{model}.yaml -loop_smpl 100 -loop_cloth 200 -colab -gpu 0 -export_video -in_dir {image_folder}
clear_output()

In [ ]:
import IPython
from IPython.display import Image
from IPython.core.display import Video
import base64
from base64 import b64encode
from IPython.display import HTML

video_path = f"/content/ICON/results/{model}/vid/{filename[:-4]}_cloth.mp4"
video_display_path = f"/content/ICON/results/{model}/vid/{filename[:-4]}_display.mp4"

def video(path):
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML('<video width=512 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

# we cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)


# Share your result! 
Please share your results with[ #icon](https://twitter.com/search?q=%23icon&src=recent_search_click&f=live) tag on Twitter. Sharing your good/bad results helps and encourages the authors to further push towards producition-quality human digitization at home.
**As the tweet buttom below doesn't add the result video automatically, please download the result video above and manually add it to the tweet.**

In [ ]:
import IPython
IPython.display.HTML('<a href="https://twitter.com/intent/tweet?button_hashtag=icon&ref_src=twsrc%5Etfw" class="twitter-hashtag-button" data-size="large" data-text="Google Colab Link: " data-url="https://bit.ly/3HysRoL" data-show-count="false">Tweet #icon</a><script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>  (Don\'t forget to add your result to the tweet!)')